### Google Problem Self-driving rides

#### Let's do the imports

In [1]:
import numpy as np
import os

In [2]:
#input_file = 'a_example.in'
#input_file = 'b_should_be_easy.in'
#input_file = 'c_no_hurry.in'
#input_file = 'd_metropolis.in'
input_file = 'e_high_bonus.in'

In [3]:
npdata = np.loadtxt(input_file, dtype=int, delimiter=' ')

In [4]:
npdata.shape

(10001, 6)

In [5]:
print(npdata[0:10,])

[[  1500   2000    350  10000   1000 150000]
 [   295   1043    796   1955  38684  51050]
 [   434    467    200    833  89213  91692]
 [  1473   1815   1321    810  17720  26914]
 [  1210    150   1287     12  23916  24939]
 [   814    563   1341    419  21032  26079]
 [  1037   1432     51    805  29678  33707]
 [  1164    491     45    767 119516 121962]
 [   519    215   1454    449 105332 113663]
 [   708   1681   1080    701 100612 107764]]


In [6]:
# Get the data of the input file's first line 
grid_rows = npdata[0,0]
grid_cols = npdata[0,1]
vehicles = npdata[0,2]
rides = npdata[0,3]
bonus_per_ride = npdata[0,4]
steps = npdata[0,5]
print("rows", grid_rows, "col", grid_cols, "vehicles", vehicles, "rides", rides, "bonus",bonus_per_ride, "steps", steps)

rows 1500 col 2000 vehicles 350 rides 10000 bonus 1000 steps 150000


In [7]:
def find_distance(start, end):
    dist = np.subtract(end,start)
    dist = np.abs(dist)
    return np.sum(dist)

In [8]:
# create array for vehicles.2 cols for the coordinates of their position and 1 for number of rides assigned to it
# 1 for the last step is occupied
vehs = np.zeros((vehicles, 4), dtype=int) # the initial position of the vehs

In [9]:
vehs.shape

(350, 4)

In [10]:
print(vehs[-5:,])

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [11]:
# array for the score of each ride col[0], col[1] to show if it has been assigned > -1 or not -1, 
#col[2]started on time > -1 or not -1, col[3] finished in time > -1 or not -1, 
#col[4] step ride started with the veh of col[1], col[5] step ride finished with the veh of col[1]
ride_score = np.ones((rides, 6), dtype=int)
ride_score *= -1

In [12]:
ride_score.shape

(10000, 6)

In [13]:
# calculate the score each ride can give, and populate the array
for i in range(1, rides + 1):
    s = np.array([npdata[i,0],npdata[i,1]])
    e = np.array([npdata[i,2],npdata[i,3]])
    ride_score[i - 1, 0] =  find_distance(s, e)
print(ride_score[0:9,]) #last 5 rides
max_score = np.max(ride_score[:,0])
min_score = np.min(ride_score[:,0])
mean_score = int(np.mean(ride_score[:,0]))
print("max score", max_score )   #max score
print("min score",  min_score) #min score
print("mean score", mean_score)

[[1413   -1   -1   -1   -1   -1]
 [ 600   -1   -1   -1   -1   -1]
 [1157   -1   -1   -1   -1   -1]
 [ 215   -1   -1   -1   -1   -1]
 [ 671   -1   -1   -1   -1   -1]
 [1613   -1   -1   -1   -1   -1]
 [1395   -1   -1   -1   -1   -1]
 [1169   -1   -1   -1   -1   -1]
 [1352   -1   -1   -1   -1   -1]]
max score 3190
min score 14
mean score 1160


In [14]:
def check_availiability(vehicle, from_step, to_step):
    test_array = ride_score[ride_score[:,1] == vehicle]
    num = test_array.shape[0]
    for i in range(num):
        cr1 = from_step > test_array[i,4] and from_step < test_array[i,5]
        cr2 = to_step > test_array[i,4] and to_step < test_array[i,5]
        cr3 = from_step < test_array[i,4] and to_step > test_array[i,5]
        if cr1 or cr2 or cr3:
            #print("not availiable")
            return False
    return True

In [15]:
def find_last_step(vehicle):
    test_array = ride_score[ride_score[:,1] == vehicle]
    num = test_array.shape[0]
    last_step = 0
    for i in range(num):
        if test_array[i,5] > last_step:
            last_step = test_array[i,5]
    return last_step

In [16]:
def find_nearest_veh(start):
    nearrest = vehicles
    nearest_distance = steps
    for veh in range(vehicles):
        position = np.array([vehs[veh, 0], vehs[veh, 1]])
        distance = find_distance(position, start)
        if distance <= nearest_distance:
            nearrest = veh
            nearest_distance = distance
    return veh
            

In [17]:
def find_0_rides_veh(less_rides):
    #less_rides = int(rides / vehicles) - 1
    a = vehicles
    i = np.where(vehs[:,2] <= less_rides)
    avail = i[0] # first availiable veh
    if len(avail) > 0:
        first_veh = avail[0]
        return first_veh
    else:
        return a

In [18]:
def record_assingment(ride, veh, bonus_start, up_to, delay):
    end = np.array([npdata[ride + 1, 2], npdata[ride + 1, 3]])
    ride_score[ride, 1] = veh
    ride_score[ride, 2] = delay # started on time = 0 
    ride_score[ride, 3] = npdata[ride + 1, 5] - up_to # Finished in time > 0 or not < 0
    ride_score[ride, 4] = bonus_start
    ride_score[ride, 5] = up_to 
    vehs[veh, 0] = end[0]
    vehs[veh, 1] = end[1]
    vehs[veh, 2] += 1
    vehs[veh, 3] = up_to

In [19]:
def do_first_assignments(rides_per_veh):
    for ride in range(rides):
        if ride_score[ride, 1] == -1: # the ride is unassigned
            start = np.array([npdata[ride + 1, 0], npdata[ride + 1, 1]])
            end = np.array([npdata[ride + 1, 2], npdata[ride + 1, 3]])
            veh = find_0_rides_veh(rides_per_veh)
            if veh == vehicles:
                return
            position = np.array([vehs[veh, 0], vehs[veh, 1]])
            steps_to_start = find_distance(position, start)
            bonus_start = npdata[ride + 1, 4] - steps_to_start
            if bonus_start < 0:
                bonus_start = 0
            delay = bonus_start - npdata[ride + 1, 4] + steps_to_start 
            steps_to_end = steps_to_start + ride_score[ride, 0]
            up_to = bonus_start + steps_to_end # step where ride will finish
            criterion0 = npdata[ride + 1, 5] - up_to > 0 # to finish in time
            criterion1 = check_availiability(veh, bonus_start, up_to)
            if criterion0 and criterion1:
                record_assingment(ride, veh, bonus_start, up_to, delay)
                print("Just assigned ride", ride, "to vehicle", veh)
                

In [20]:
for i in range(int(rides / vehicles)):
    do_first_assignments(i)

Just assigned ride 0 to vehicle 0
Just assigned ride 1 to vehicle 1
Just assigned ride 2 to vehicle 2
Just assigned ride 3 to vehicle 3
Just assigned ride 4 to vehicle 4
Just assigned ride 5 to vehicle 5
Just assigned ride 6 to vehicle 6
Just assigned ride 7 to vehicle 7
Just assigned ride 8 to vehicle 8
Just assigned ride 9 to vehicle 9
Just assigned ride 10 to vehicle 10
Just assigned ride 11 to vehicle 11
Just assigned ride 12 to vehicle 12
Just assigned ride 13 to vehicle 13
Just assigned ride 14 to vehicle 14
Just assigned ride 15 to vehicle 15
Just assigned ride 16 to vehicle 16
Just assigned ride 17 to vehicle 17
Just assigned ride 18 to vehicle 18
Just assigned ride 19 to vehicle 19
Just assigned ride 20 to vehicle 20
Just assigned ride 21 to vehicle 21
Just assigned ride 22 to vehicle 22
Just assigned ride 23 to vehicle 23
Just assigned ride 24 to vehicle 24
Just assigned ride 25 to vehicle 25
Just assigned ride 26 to vehicle 26
Just assigned ride 27 to vehicle 27
Just assigne

Just assigned ride 287 to vehicle 287
Just assigned ride 288 to vehicle 288
Just assigned ride 289 to vehicle 289
Just assigned ride 290 to vehicle 290
Just assigned ride 291 to vehicle 291
Just assigned ride 292 to vehicle 292
Just assigned ride 293 to vehicle 293
Just assigned ride 294 to vehicle 294
Just assigned ride 295 to vehicle 295
Just assigned ride 296 to vehicle 296
Just assigned ride 297 to vehicle 297
Just assigned ride 298 to vehicle 298
Just assigned ride 299 to vehicle 299
Just assigned ride 300 to vehicle 300
Just assigned ride 301 to vehicle 301
Just assigned ride 302 to vehicle 302
Just assigned ride 303 to vehicle 303
Just assigned ride 304 to vehicle 304
Just assigned ride 305 to vehicle 305
Just assigned ride 306 to vehicle 306
Just assigned ride 307 to vehicle 307
Just assigned ride 308 to vehicle 308
Just assigned ride 309 to vehicle 309
Just assigned ride 310 to vehicle 310
Just assigned ride 311 to vehicle 311
Just assigned ride 312 to vehicle 312
Just assigne

Just assigned ride 555 to vehicle 197
Just assigned ride 556 to vehicle 198
Just assigned ride 557 to vehicle 199
Just assigned ride 558 to vehicle 200
Just assigned ride 559 to vehicle 201
Just assigned ride 560 to vehicle 202
Just assigned ride 561 to vehicle 203
Just assigned ride 562 to vehicle 204
Just assigned ride 563 to vehicle 205
Just assigned ride 564 to vehicle 206
Just assigned ride 565 to vehicle 207
Just assigned ride 566 to vehicle 208
Just assigned ride 567 to vehicle 209
Just assigned ride 569 to vehicle 210
Just assigned ride 570 to vehicle 211
Just assigned ride 571 to vehicle 212
Just assigned ride 572 to vehicle 213
Just assigned ride 573 to vehicle 214
Just assigned ride 574 to vehicle 215
Just assigned ride 575 to vehicle 216
Just assigned ride 576 to vehicle 217
Just assigned ride 577 to vehicle 218
Just assigned ride 578 to vehicle 219
Just assigned ride 579 to vehicle 220
Just assigned ride 580 to vehicle 221
Just assigned ride 581 to vehicle 222
Just assigne

Just assigned ride 815 to vehicle 112
Just assigned ride 816 to vehicle 113
Just assigned ride 817 to vehicle 114
Just assigned ride 818 to vehicle 115
Just assigned ride 819 to vehicle 116
Just assigned ride 820 to vehicle 117
Just assigned ride 821 to vehicle 118
Just assigned ride 822 to vehicle 119
Just assigned ride 823 to vehicle 120
Just assigned ride 824 to vehicle 121
Just assigned ride 825 to vehicle 122
Just assigned ride 826 to vehicle 123
Just assigned ride 827 to vehicle 124
Just assigned ride 828 to vehicle 125
Just assigned ride 829 to vehicle 126
Just assigned ride 830 to vehicle 127
Just assigned ride 831 to vehicle 128
Just assigned ride 832 to vehicle 129
Just assigned ride 833 to vehicle 130
Just assigned ride 834 to vehicle 131
Just assigned ride 835 to vehicle 132
Just assigned ride 836 to vehicle 133
Just assigned ride 837 to vehicle 134
Just assigned ride 838 to vehicle 135
Just assigned ride 839 to vehicle 136
Just assigned ride 840 to vehicle 137
Just assigne

Just assigned ride 920 to vehicle 5
Just assigned ride 934 to vehicle 6
Just assigned ride 958 to vehicle 7
Just assigned ride 1001 to vehicle 8
Just assigned ride 1009 to vehicle 9
Just assigned ride 1013 to vehicle 10
Just assigned ride 1014 to vehicle 11
Just assigned ride 1023 to vehicle 12
Just assigned ride 1038 to vehicle 13
Just assigned ride 1050 to vehicle 14
Just assigned ride 1055 to vehicle 15
Just assigned ride 1068 to vehicle 16
Just assigned ride 1069 to vehicle 17
Just assigned ride 1070 to vehicle 18
Just assigned ride 1071 to vehicle 19
Just assigned ride 1072 to vehicle 20
Just assigned ride 1073 to vehicle 21
Just assigned ride 1074 to vehicle 22
Just assigned ride 1075 to vehicle 23
Just assigned ride 1076 to vehicle 24
Just assigned ride 1079 to vehicle 25
Just assigned ride 1080 to vehicle 26
Just assigned ride 1081 to vehicle 27
Just assigned ride 1082 to vehicle 28
Just assigned ride 1083 to vehicle 29
Just assigned ride 1084 to vehicle 30
Just assigned ride 1

...

Just assigned ride 8893 to vehicle 315
Just assigned ride 8895 to vehicle 316
Just assigned ride 8907 to vehicle 317
Just assigned ride 8913 to vehicle 318
Just assigned ride 8914 to vehicle 319
Just assigned ride 8916 to vehicle 320
Just assigned ride 8922 to vehicle 321
Just assigned ride 8925 to vehicle 322
Just assigned ride 8926 to vehicle 323
Just assigned ride 8931 to vehicle 324
Just assigned ride 8935 to vehicle 325
Just assigned ride 8938 to vehicle 326
Just assigned ride 8940 to vehicle 327
Just assigned ride 8949 to vehicle 328
Just assigned ride 8950 to vehicle 329
Just assigned ride 8955 to vehicle 330
Just assigned ride 8965 to vehicle 331
Just assigned ride 8966 to vehicle 332
Just assigned ride 8967 to vehicle 333
Just assigned ride 8968 to vehicle 334
Just assigned ride 8972 to vehicle 335
Just assigned ride 8974 to vehicle 336
Just assigned ride 8978 to vehicle 337
Just assigned ride 8979 to vehicle 338
Just assigned ride 8980 to vehicle 339
Just assigned ride 8991 t

Just assigned ride 9128 to vehicle 233
Just assigned ride 9129 to vehicle 234
Just assigned ride 9131 to vehicle 235
Just assigned ride 9133 to vehicle 236
Just assigned ride 9137 to vehicle 237
Just assigned ride 9138 to vehicle 238
Just assigned ride 9141 to vehicle 239
Just assigned ride 9143 to vehicle 240
Just assigned ride 9145 to vehicle 241
Just assigned ride 9147 to vehicle 242
Just assigned ride 9148 to vehicle 243
Just assigned ride 9149 to vehicle 244
Just assigned ride 9150 to vehicle 245
Just assigned ride 9152 to vehicle 246
Just assigned ride 9153 to vehicle 247
Just assigned ride 9158 to vehicle 248
Just assigned ride 9159 to vehicle 249
Just assigned ride 9160 to vehicle 250
Just assigned ride 9162 to vehicle 251
Just assigned ride 9164 to vehicle 252
Just assigned ride 9171 to vehicle 253
Just assigned ride 9172 to vehicle 254
Just assigned ride 9174 to vehicle 255
Just assigned ride 9180 to vehicle 256
Just assigned ride 9181 to vehicle 257
Just assigned ride 9182 t

Just assigned ride 9232 to vehicle 165
Just assigned ride 9234 to vehicle 166
Just assigned ride 9241 to vehicle 167
Just assigned ride 9245 to vehicle 168
Just assigned ride 9256 to vehicle 169
Just assigned ride 9267 to vehicle 170
Just assigned ride 9278 to vehicle 171
Just assigned ride 9280 to vehicle 172
Just assigned ride 9283 to vehicle 173
Just assigned ride 9294 to vehicle 174
Just assigned ride 9296 to vehicle 175
Just assigned ride 9301 to vehicle 176
Just assigned ride 9304 to vehicle 177
Just assigned ride 9311 to vehicle 178
Just assigned ride 9317 to vehicle 179
Just assigned ride 9318 to vehicle 180
Just assigned ride 9319 to vehicle 181
Just assigned ride 9320 to vehicle 182
Just assigned ride 9326 to vehicle 183
Just assigned ride 9335 to vehicle 184
Just assigned ride 9344 to vehicle 185
Just assigned ride 9354 to vehicle 186
Just assigned ride 9357 to vehicle 187
Just assigned ride 9359 to vehicle 188
Just assigned ride 9362 to vehicle 189
Just assigned ride 9368 t

Just assigned ride 8951 to vehicle 74
Just assigned ride 8953 to vehicle 75
Just assigned ride 8983 to vehicle 76
Just assigned ride 8997 to vehicle 77
Just assigned ride 8999 to vehicle 78
Just assigned ride 9013 to vehicle 79
Just assigned ride 9014 to vehicle 80
Just assigned ride 9024 to vehicle 81
Just assigned ride 9025 to vehicle 82
Just assigned ride 9068 to vehicle 83
Just assigned ride 9081 to vehicle 84
Just assigned ride 9092 to vehicle 85
Just assigned ride 9120 to vehicle 86
Just assigned ride 9123 to vehicle 87
Just assigned ride 9136 to vehicle 88
Just assigned ride 9151 to vehicle 89
Just assigned ride 9173 to vehicle 90
Just assigned ride 9179 to vehicle 91
Just assigned ride 9189 to vehicle 92
Just assigned ride 9193 to vehicle 93
Just assigned ride 9206 to vehicle 94
Just assigned ride 9208 to vehicle 95
Just assigned ride 9210 to vehicle 96
Just assigned ride 9212 to vehicle 97
Just assigned ride 9213 to vehicle 98
Just assigned ride 9219 to vehicle 99
Just assigne

Just assigned ride 9409 to vehicle 75
Just assigned ride 9412 to vehicle 76
Just assigned ride 9415 to vehicle 77
Just assigned ride 9473 to vehicle 78
Just assigned ride 9478 to vehicle 79
Just assigned ride 9483 to vehicle 80
Just assigned ride 9488 to vehicle 81
Just assigned ride 9490 to vehicle 82
Just assigned ride 9491 to vehicle 83
Just assigned ride 9496 to vehicle 84
Just assigned ride 9498 to vehicle 85
Just assigned ride 9505 to vehicle 86
Just assigned ride 9522 to vehicle 87
Just assigned ride 9527 to vehicle 88
Just assigned ride 9553 to vehicle 89
Just assigned ride 9555 to vehicle 90
Just assigned ride 9578 to vehicle 91
Just assigned ride 9580 to vehicle 92
Just assigned ride 9582 to vehicle 93
Just assigned ride 9585 to vehicle 94
Just assigned ride 9601 to vehicle 95
Just assigned ride 9606 to vehicle 96
Just assigned ride 9616 to vehicle 97
Just assigned ride 9618 to vehicle 98
Just assigned ride 9629 to vehicle 99
Just assigned ride 9632 to vehicle 100
Just assign

In [21]:
unassigned = len(ride_score[ride_score[:,1 ] == -1])
print("There are ", unassigned, "rides unassigned")

There are  498 rides unassigned


In [22]:
os.system('spd-say "your first assignments have finished"')

0

In [23]:
def assign_rides(ratio):
    for ride in range(rides):
        start = np.array([npdata[ride + 1, 0], npdata[ride + 1, 1]])
        end = np.array([npdata[ride + 1, 2], npdata[ride + 1, 3]])
        for veh in range(vehicles):
            position = np.array([vehs[veh, 0], vehs[veh, 1]])
            steps_to_start = find_distance(position, start)
            bonus_start = npdata[ride + 1, 4] - steps_to_start
            steps_to_end = steps_to_start + ride_score[ride, 0]
            up_to = bonus_start + steps_to_end # step where ride will finish
            delay = 0
            if not check_availiability(veh, bonus_start, up_to):
                bonus_start = vehs[veh, 3] + 1
                up_to = bonus_start + steps_to_end
                delay = bonus_start - npdata[ride + 1, 4] + steps_to_start
            criterion1 = npdata[ride + 1, 5] - up_to > 0 # to finish in time
            criterion0 = bonus_start >= 0 # start at a step >= 0
            #criterion1 = check_availiability(veh, bonus_start, up_to)
            criterion2 = ride_score[ride, 1] == -1 # the ride is unassigned
            if vehs[veh, 2] > 0:
                criterion3 = (rides / (vehicles * vehs[veh, 2]) > ratio)  # for better distribution
            else:
                criterion3 = True              
            if criterion0 and criterion1 and criterion2 and criterion3:
                print("Just assigned ride", ride, "to vehicle", veh)
                record_assingment(ride, veh, bonus_start, up_to, delay)
                break

In [24]:
assign_rides(0.9)

Just assigned ride 2210 to vehicle 0
Just assigned ride 2542 to vehicle 3
Just assigned ride 3171 to vehicle 5
Just assigned ride 3278 to vehicle 3
Just assigned ride 3311 to vehicle 1
Just assigned ride 3662 to vehicle 2
Just assigned ride 4270 to vehicle 0
Just assigned ride 4359 to vehicle 2
Just assigned ride 4640 to vehicle 3
Just assigned ride 4763 to vehicle 2
Just assigned ride 5116 to vehicle 3
Just assigned ride 5127 to vehicle 4
Just assigned ride 5130 to vehicle 5
Just assigned ride 5148 to vehicle 4
Just assigned ride 5151 to vehicle 4
Just assigned ride 5233 to vehicle 4
Just assigned ride 5436 to vehicle 5
Just assigned ride 5532 to vehicle 6
Just assigned ride 5578 to vehicle 5
Just assigned ride 5657 to vehicle 0
Just assigned ride 5759 to vehicle 6
Just assigned ride 5778 to vehicle 2
Just assigned ride 5786 to vehicle 6
Just assigned ride 6194 to vehicle 6
Just assigned ride 6338 to vehicle 7
Just assigned ride 6351 to vehicle 10
Just assigned ride 6437 to vehicle 10

Just assigned ride 9175 to vehicle 57
Just assigned ride 9177 to vehicle 54
Just assigned ride 9178 to vehicle 57
Just assigned ride 9184 to vehicle 56
Just assigned ride 9185 to vehicle 56
Just assigned ride 9187 to vehicle 54
Just assigned ride 9195 to vehicle 54
Just assigned ride 9205 to vehicle 56
Just assigned ride 9207 to vehicle 55
Just assigned ride 9211 to vehicle 59
Just assigned ride 9214 to vehicle 55
Just assigned ride 9217 to vehicle 56
Just assigned ride 9218 to vehicle 58
Just assigned ride 9220 to vehicle 59
Just assigned ride 9225 to vehicle 55
Just assigned ride 9229 to vehicle 58
Just assigned ride 9231 to vehicle 58
Just assigned ride 9235 to vehicle 58
Just assigned ride 9236 to vehicle 63
Just assigned ride 9239 to vehicle 55
Just assigned ride 9242 to vehicle 63
Just assigned ride 9244 to vehicle 59
Just assigned ride 9255 to vehicle 59
Just assigned ride 9258 to vehicle 60
Just assigned ride 9260 to vehicle 60
Just assigned ride 9264 to vehicle 60
Just assigne

Just assigned ride 9873 to vehicle 106
Just assigned ride 9874 to vehicle 108
Just assigned ride 9875 to vehicle 108
Just assigned ride 9878 to vehicle 109
Just assigned ride 9880 to vehicle 109
Just assigned ride 9881 to vehicle 109
Just assigned ride 9884 to vehicle 110
Just assigned ride 9885 to vehicle 110
Just assigned ride 9887 to vehicle 112
Just assigned ride 9890 to vehicle 111
Just assigned ride 9893 to vehicle 110
Just assigned ride 9898 to vehicle 111
Just assigned ride 9899 to vehicle 111
Just assigned ride 9900 to vehicle 110
Just assigned ride 9901 to vehicle 111
Just assigned ride 9903 to vehicle 112
Just assigned ride 9904 to vehicle 112
Just assigned ride 9905 to vehicle 112
Just assigned ride 9906 to vehicle 113
Just assigned ride 9909 to vehicle 116
Just assigned ride 9910 to vehicle 113
Just assigned ride 9914 to vehicle 113
Just assigned ride 9915 to vehicle 113
Just assigned ride 9916 to vehicle 114
Just assigned ride 9918 to vehicle 114
Just assigned ride 9921 t

In [25]:
os.system('spd-say "your first pass has finished"')

0

In [26]:
unassigned = len(ride_score[ride_score[:,1 ] == -1])
print("There are ", unassigned, "rides unassigned")

There are  0 rides unassigned


In [27]:
def assign_rides_second_pass():
    for ride in range(rides):
        #print(ride, ride_score[ride, 1])
        if ride_score[ride, 1] == -1: # the ride is unassigned
            start = np.array([npdata[ride + 1, 0], npdata[ride + 1, 1]])
            end = np.array([npdata[ride + 1, 2], npdata[ride + 1, 3]])
            #veh = find_nearest_veh(start)
            veh = find_0_rides_veh(0)
            if veh == vehicles:
                return
            position = np.array([vehs[veh, 0], vehs[veh, 1]])
            steps_to_start = find_distance(position, start)
            bonus_start = npdata[ride + 1, 4] - steps_to_start
            if bonus_start < 0:
                bonus_start = 0
            delay = bonus_start - npdata[ride + 1, 4] + steps_to_start 
            steps_to_end = steps_to_start + ride_score[ride, 0]
            up_to = bonus_start + steps_to_end # step where ride will finish
            criterion0 = npdata[ride + 1, 5] - up_to > 0 # to finish in time 
            if criterion0 :
                record_assingment(ride, veh, bonus_start, up_to, delay)
                print("Just assigned ride", ride, "to vehicle", veh)
                

In [28]:
if unassigned > 0:
    assign_rides_second_pass()

In [29]:
os.system('spd-say "your second pass has finished"')

0

In [30]:
unassigned = len(ride_score[ride_score[:,1 ] == -1])
print("There are ", unassigned, "rides unassigned")

There are  0 rides unassigned


In [31]:
def assign_rides_third_pass(ratio):
    for ride in range(rides):
        #print(ride, ride_score[ride, 1])
        if ride_score[ride, 1] == -1: # the ride is unassigned
            start = np.array([npdata[ride + 1, 0], npdata[ride + 1, 1]])
            end = np.array([npdata[ride + 1, 2], npdata[ride + 1, 3]])
            veh = find_nearest_veh(start)
            position = np.array([vehs[veh, 0], vehs[veh, 1]])
            steps_to_start = find_distance(position, start)
            bonus_start = vehs[veh, 2] + 1
            delay = bonus_start - npdata[ride + 1, 4] + steps_to_start 
            steps_to_end = steps_to_start + ride_score[ride, 0]
            up_to = bonus_start + steps_to_end # step where ride will finish
            criterion0 = npdata[ride + 1, 5] - up_to > 0 # to finish in time
            if vehs[veh, 2] > 0:
                criterion3 = (rides / (vehicles * vehs[veh, 2]) > ratio)  # for better distribution
            else:
                criterion3 = True    
            if criterion0 and criterion3:
                record_assingment(ride, veh, bonus_start, up_to, delay)
                print("Just assigned ride", ride, "to vehicle", veh)
                

In [32]:
count = 0
while -1 in ride_score[:,1] and count < 3:
    assign_rides_third_pass(0.3)
    count += 1

In [33]:
os.system('spd-say "your third pass has finished"')

0

In [34]:
unassigned = len(ride_score[ride_score[:,1 ] == -1])
print("There are ", unassigned, "rides unassigned")

There are  0 rides unassigned


In [35]:
def write_output(file):
    try:
        output_file = open(file, "w")
    except:
        output_file.close()
        return "Can not open the " + file
    for veh in range(vehicles):
        output_string = str(vehs[veh,2])
        test_array = ride_score[ride_score[:,1] == veh]
        if test_array.shape[0] != vehs[veh,2]:
            return "ERROR"
        for i in range(rides):
            if ride_score[i,1] == veh:
                output_string = output_string + " " + str(i)
        print(output_string, file = output_file)
        print(output_string)
    output_file.close()

In [36]:
out_file = "sub1" + input_file
write_output(out_file)

32 0 350 377 731 884 1098 1493 1625 1902 1974 2210 2544 2581 3108 3268 3412 3846 4134 4186 4270 4439 4611 4757 5199 5657 5744 6077 6221 6342 6412 6573 6646
32 1 351 407 780 1067 1108 1528 1916 2020 2045 2640 2746 3118 3270 3311 3906 4017 4137 4551 4662 5028 5036 5538 5779 6155 6304 6439 6762 6769 6777 6797 6852
32 2 352 424 803 1077 1159 1529 1963 2042 2232 2648 2820 3300 3411 3662 3992 4021 4247 4359 4746 4763 4822 5234 5443 5668 5778 5874 6377 6569 6708 6830 7041
32 3 353 441 862 1078 1194 1582 2019 2236 2444 2542 2747 2889 3278 3333 3648 4042 4064 4377 4640 4938 5116 5131 5256 5508 5741 5909 6378 6786 6814 7038 7158
32 4 354 442 911 1092 1252 1603 2081 2290 2511 2941 3096 3405 3722 4072 4075 4385 4989 5127 5148 5151 5198 5233 5435 5621 5742 5991 6475 6899 6904 7112 7496
32 5 355 465 920 1096 1412 1651 2109 2308 2541 2950 3159 3171 3469 3775 4094 4152 4426 5034 5130 5200 5436 5534 5578 5746 5787 5994 6534 6914 7117 7183 7576
32 6 356 535 934 1130 1452 1672 2113 2329 2556 2956 3246 34

32 64 416 765 1122 1472 1832 2179 2535 2825 3220 3452 3849 4114 4532 4896 5165 5596 5975 6124 6639 6876 7228 7450 7899 8203 8475 8806 9306 9316 9343 9353 9374
32 65 417 766 1123 1473 1833 2180 2536 2826 3225 3456 3851 4115 4534 4899 5173 5599 5976 6139 6644 6889 7233 7462 7902 8214 8482 8812 9297 9307 9323 9377 9379
32 66 418 767 1124 1474 1834 2181 2538 2827 3236 3464 3860 4117 4537 4901 5184 5601 5979 6142 6651 6890 7237 7467 7912 8227 8500 8830 9314 9325 9327 9352 9384
32 67 419 768 1125 1475 1835 2182 2540 2835 3237 3466 3865 4121 4538 4919 5186 5609 5983 6159 6678 6896 7239 7482 7916 8261 8505 8836 9315 9387 9406 9411 9426
32 68 420 769 1126 1476 1837 2183 2545 2839 3247 3467 3866 4123 4539 4928 5187 5610 5986 6175 6679 6935 7242 7505 7923 8268 8516 8844 9321 9395 9414 9431 9432
32 69 421 770 1127 1477 1838 2184 2547 2854 3253 3474 3877 4135 4552 4933 5188 5615 5987 6176 6682 6937 7247 7513 7924 8276 8543 8846 9329 9397 9433 9438 9444
32 70 422 771 1128 1478 1839 2185 2548 2856 32

28 138 494 841 1205 1556 1920 2269 2634 2977 3355 3702 4058 4415 4778 5150 5419 5860 6260 6533 6982 7266 7621 8001 8362 8835 9089 9525 9895
28 139 495 842 1206 1558 1921 2271 2635 2978 3356 3703 4059 4416 4779 5152 5431 5861 6261 6538 6986 7270 7622 8003 8365 8843 9090 9535 9907
28 140 496 843 1207 1559 1922 2272 2636 2979 3357 3704 4060 4417 4780 5153 5434 5872 6262 6541 6989 7277 7630 8005 8368 8851 9094 9552 9911
28 141 497 844 1208 1560 1923 2273 2637 2980 3359 3705 4061 4418 4781 5155 5440 5876 6264 6543 6995 7280 7633 8015 8373 8855 9102 9560 9920
28 142 498 845 1209 1561 1925 2274 2638 2981 3360 3707 4062 4421 4783 5157 5442 5879 6266 6547 7000 7283 7635 8018 8381 8858 9103 9564 9926
28 143 499 846 1210 1563 1926 2275 2641 2982 3361 3708 4063 4422 4785 5158 5464 5880 6267 6548 7002 7286 7636 8026 8389 8859 9106 9566 9928
28 144 500 847 1211 1564 1927 2276 2642 2983 3362 3709 4066 4423 4788 5160 5469 5882 6273 6552 7008 7289 7649 8028 8396 8863 9107 9569 9930
28 145 501 848 1212 

27 216 575 923 1293 1643 2010 2362 2736 3085 3463 3814 4184 4533 4907 5294 5618 6000 6399 6754 7175 7554 7901 8292 8661 9086 9464 9912
27 217 576 924 1294 1644 2011 2364 2737 3088 3465 3815 4187 4535 4908 5298 5619 6001 6400 6761 7176 7555 7903 8294 8662 9091 9466 9913
27 218 577 925 1295 1645 2012 2365 2738 3089 3468 3817 4188 4536 4910 5299 5620 6002 6403 6763 7177 7557 7905 8295 8663 9093 9467 9917
27 219 578 926 1296 1646 2013 2366 2739 3090 3470 3818 4189 4540 4911 5300 5628 6003 6406 6766 7179 7560 7906 8305 8667 9096 9470 9919
27 220 579 927 1297 1647 2014 2367 2740 3092 3471 3820 4190 4541 4912 5303 5629 6004 6407 6770 7184 7562 7907 8306 8671 9098 9475 9923
27 221 580 928 1298 1649 2015 2368 2743 3093 3472 3821 4191 4542 4913 5304 5632 6005 6408 6771 7186 7563 7910 8316 8673 9099 9476 9924
27 222 581 929 1299 1650 2016 2369 2744 3097 3473 3822 4192 4543 4914 5305 5634 6007 6410 6776 7187 7564 7913 8318 8674 9100 9477 9927
27 223 582 930 1301 1652 2017 2371 2745 3098 3475 3823 

26 292 655 1002 1384 1735 2106 2454 2832 3187 3560 3915 4292 4649 5037 5424 5771 6136 6537 6886 7332 7725 8073 8498 8832 9293 9688
26 293 656 1003 1385 1736 2107 2455 2833 3188 3562 3916 4293 4650 5038 5425 5772 6137 6540 6887 7333 7728 8074 8499 8834 9295 9692
26 294 657 1004 1386 1737 2108 2456 2834 3189 3563 3917 4294 4651 5039 5426 5773 6138 6544 6888 7335 7729 8076 8504 8837 9298 9700
26 295 658 1005 1387 1738 2110 2457 2836 3191 3564 3919 4295 4653 5043 5427 5774 6141 6545 6894 7336 7730 8077 8506 8840 9299 9702
26 296 659 1006 1388 1739 2111 2458 2837 3192 3565 3922 4297 4654 5044 5428 5775 6143 6550 6897 7338 7731 8079 8510 8842 9302 9703
26 297 660 1007 1389 1740 2112 2459 2838 3193 3566 3923 4299 4655 5048 5429 5776 6144 6553 6898 7340 7734 8080 8515 8845 9305 9707
26 298 661 1008 1390 1741 2114 2460 2840 3194 3567 3924 4300 4658 5049 5430 5777 6145 6554 6902 7348 7738 8083 8517 8847 9308 9711
26 299 662 1010 1391 1742 2115 2461 2841 3195 3568 3925 4301 4659 5050 5432 5783 61

In [37]:
def calculate_score():
    mask = ride_score[:,3] > 0 #to take into account only the assigned rides finished in time
    final_score = np.sum(ride_score[mask], axis = 0)
    total_bonus = bonus_per_ride * len(ride_score[ride_score[:,2 ]== 0])
    print("final score =", final_score[0])
    print("finished in time", np.sum(ride_score[:,3 ] >= 0))
    print("Got bonus for ", len(ride_score[ride_score[:,2 ]== 0]))
    print("total bonus", total_bonus)
    total_score = final_score[0] + total_bonus
    print("total score = ", total_score)

In [38]:
calculate_score()

final score = 11601343
finished in time 10000
Got bonus for  9610
total bonus 9610000
total score =  21211343


In [39]:
os.system('spd-say "your program has finished. Check the results"')

0